In [2]:
import pandas
import pandas as pd
import numpy as np
import sys
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tqdm import tqdm

deepctr_path = '/root/linghui/rec-fairness/'
sys.path.append(deepctr_path)
import deepctr
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, get_feature_names

2022-07-04 14:21:17.694765: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
DeepCTR version 0.9.1 detected. Your version is 0.9.0.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.9.1


In [20]:
import sys
import tensorflow as tf
deepctr_path = '/root/linghui/rec-fairness/'
sys.path.append(deepctr_path)
import deepctr
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, get_feature_names

def get_feat_dict(data):
    if data == 'ml-1m':
        feat_dict = {'movie_id': 3706,
                 'user_id': 6040,
                 'gender': 2,
                 'age': 7,
                 'occupation': 21,
                 'zipcode': 3439}
    if data == 'black-fri':
        feat_dict = {'user_id': 5891,
                 'item_id': 3631,
                 'gender': 2,
                 'age': 7,
                 'occupation': 21,
                 'city': 3,
                 'stay_years': 5,
                 'martial_status': 2,
                 'cate': 20,
                 'subcate_1': 18,
                 'subcate_2': 16,
                 'price': 23962}
    return feat_dict

def get_saved_model_details(model_params, data_params):
    model_name = model_params['model_name']
    cate = model_params['cate']
    dataset = data_params['dataset']
    if dataset == 'ml-1m':
        if model_name == 'DeepFM':
            if cate == 'all-feat':
                check_path = '/data/linghui/saved_model/deepfm-ml-1m/deepfm-all-feature/deepfm-ml-1m.ckpt'
                train_feats = ["movie_id", "user_id", "gender", "age", "occupation", "zipcode"]
            if cate == 'del-sf':
                check_path = '/data/linghui/saved_model/deepfm-ml-1m/deepfm-del-sf/deepfm-ml-1m-del-sf.ckpt'
                train_feats = ["movie_id", "user_id"]
            if cate == 'liuyi-all-feats':
                check_path = '/data/linghui/saved_model/deepfm-ml-1m/deepfm-liuyi/deepfm-ml-1m-liuyi.ckpt'
                train_feats = ["movie_id", "user_id", "gender", "age", "occupation", "zipcode"]
    if dataset == 'black-fri':
        if model_name == 'DeepFM':
            if cate == 'liuyi-all-feats':
                check_path = '/data/linghui/saved_model/deepfm-black-fri/deepfm-black-fri-liuyi.ckpt'
                train_feats = ['user_id','item_id','gender','age','occupation','city','stay_years','martial_status','cate','subcate_1','subcate_2','price']
    return check_path , train_feats

def get_model(model_params,data_params):
    dataset = data_params['dataset']

    model_name = model_params['model_name']
    cate = model_params['cate']
    embedding_dim = model_params['embedding_dim']
    check_path, train_feats = get_saved_model_details(model_params,data_params)
    
    feat_dict = get_feat_dict(dataset)
    fixlen_feature_columns = [SparseFeat(feat, feat_dict[feat], embedding_dim=embedding_dim)
                                      for feat in train_feats]
    linear_feature_columns = fixlen_feature_columns 
    dnn_feature_columns = fixlen_feature_columns 
    feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
    

    if model_name == 'DeepFM':
        model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
    model.summary()
    model.load_weights(check_path)
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy',
                  metrics=['AUC', 'Precision', 'Recall'])
    return model


def create_single_rule(key,value):
    return{key:value}

def get_single_sparse_rule(df,features):
    single_rule = []
    feat_value = df[features].value_counts()
    feat_value = feat_value.index
    sparse_feat_value = feat_value.values.astype('int32')
    for value in sparse_feat_value:
        rule = create_single_rule(features,value)
        single_rule.append(rule)
    return single_rule

def get_single_dense_rule(df,feature,k):
    min_value = df[feature].min()
    max_value = df[feature].max()
    single_rule = []
    d = math.ceil((max_value - min_value) / k)
    for i in range(k):
        x = min_value + i * d
        y = x + d
        single_rule.append({feature:[x,y]})
    return single_rule

def merge_two_dicts(x, y):
    z = x.copy()
    z.update(y)
    return z

def combine_rule(rule_dict_1,rule_dict_2):
    len1 = len(rule_dict_1)
    len2 = len(rule_dict_2)
    all_rule = []
    for i in range(len1):
        rule1 = rule_dict_1[i]
        for j in range(len2):
            rule2 = rule_dict_2[j]
            merge_rule = merge_two_dicts(rule1,rule2)
            all_rule.append(merge_rule)
    return all_rule

def creat_sparse_rule_query(rule):
    result = ''
    key = rule.keys()
    length = len(key)
    count = 0
    for i in key:
        count = count + 1
        result = result + '( ' + i +' == ' + str(rule[i]) + ')'
        # elif i[0] == 'I':
        #     result = result + '(' + str(rule[i][0]) + ' <= ' + i + ' <= ' +  str(rule[i][1]) + ')'
        if count < length:
            result = result + ' & '
    return result

def get_all_rule_dict(data,sparse_feats,dense_feats,dense_num):
    single_rule_dict = []
    all_rule_dict = []
    if len(sparse_feats) > 0:
        for feature in sparse_feats:
            temp_rule = []
            single_rule = get_single_sparse_rule(data,feature)  
            single_rule_dict = single_rule_dict + single_rule
            if len(all_rule_dict) > 0: 
                temp_rule = combine_rule(all_rule_dict,single_rule)
            all_rule_dict = all_rule_dict + single_rule + temp_rule
    if len(dense_feats) > 0:
        for feature in dense_feats:
            temp_rule = []
            single_rule = get_single_dense_rule(data,feature,dense_num)  
            single_rule_dict = single_rule_dict + single_rule
            if len(all_rule_dict) > 0: 
                temp_rule = combine_rule(all_rule_dict,single_rule)
            all_rule_dict = all_rule_dict + single_rule + temp_rule 
    return all_rule_dict

def get_evaluate_data(data,rule):
    result = creat_sparse_rule_query(rule)
    select_group = data.query(result)
    unselect_group = data.drop(select_group.index)

    return select_group,unselect_group

def cut_rule(length_all,length_group,theta):
    x = length_group / length_all
    if (x >= theta) & (x < 1-theta):
        return 1
    else:
        return 0

def get_test_data(x_file,y_file):
    test_x = np.loadtxt(x_file)
    test_y = np.loadtxt(y_file)

    sparse_features = ['C' + str(i) for i in range(1, 27)]
    dense_features = ['I' + str(i) for i in range(1, 14)]
    features = sparse_features + dense_features

    test_x = pd.DataFrame(test_x,columns=features)
    test_y = pd.DataFrame(test_y,columns=['label'])

    test = pd.concat([test_x, test_y], axis=1)
    return test

In [15]:
saved_model_path = '/data/linghui/saved_model/deepfm-black-fri/'

saved_data_path = '/data/linghui/black-friday/processed_data/'

test_data_path = saved_data_path + 'test_data_liuyi.csv'

#load data
test_data = pd.read_csv(test_data_path,index_col=0)

test_data

,user_id,item_id,gender,age,occupation,city,stay_years,martial_status,cate,subcate_1,subcate_2,price,label,popularity
0,0,2376,0,0,10,0,2,0,0,5,11,15200,1,0.105623
1,1,1972,1,6,16,2,4,0,5,7,0,20434,1,0.099260
2,2,1079,1,2,15,0,3,0,0,1,3,3938,1,0.054175
3,3,3604,1,4,7,1,2,1,0,15,0,15686,1,0.165798
4,4,286,1,2,20,0,1,1,7,0,0,5894,1,0.013635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294545,5890,1213,1,2,6,1,2,0,10,15,0,6110,0,0.088171
294546,5890,3094,1,2,6,1,2,0,4,0,0,7095,0,0.028906
294547,5890,1824,1,2,6,1,2,0,4,13,0,3709,0,0.083808
294548,5890,3256,1,2,6,1,2,0,7,0,0,6010,0,0.001091


In [21]:
# model_name: DeepFM / 
# cate : all-feat / del-sf
model_params = {'model_name': 'DeepFM',
                'cate': 'liuyi-all-feats',
                'embedding_dim': 4,
                'batch_size': 256 }

data_params = {'dataset': 'black-fri'}

check_path ,sparse_features = get_saved_model_details(model_params, data_params)

model = get_model(model_params,data_params)


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
gender (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
age (InputLayer)                [(None, 1)]          0                                            
_______________________________________________________________________________________

In [22]:
test_input = {name: test_data[name].values for name in sparse_features}
label = test_data['label'].values
y_true = label
y_pred = model.predict(test_input)

# select_feats = ['user_id',"movie_id","rating","gender","age","occupation","zipcode"]

# evaluate_data = test_data.loc[:,select_feats]

In [23]:
test_data['predict'] = y_pred
test_data

,user_id,item_id,gender,age,occupation,city,stay_years,martial_status,cate,subcate_1,subcate_2,price,label,popularity,predict
0,0,2376,0,0,10,0,2,0,0,5,11,15200,1,0.105623,0.847192
1,1,1972,1,6,16,2,4,0,5,7,0,20434,1,0.099260,0.810600
2,2,1079,1,2,15,0,3,0,0,1,3,3938,1,0.054175,0.445903
3,3,3604,1,4,7,1,2,1,0,15,0,15686,1,0.165798,0.987372
4,4,286,1,2,20,0,1,1,7,0,0,5894,1,0.013635,0.630423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294545,5890,1213,1,2,6,1,2,0,10,15,0,6110,0,0.088171,0.766641
294546,5890,3094,1,2,6,1,2,0,4,0,0,7095,0,0.028906,0.702589
294547,5890,1824,1,2,6,1,2,0,4,13,0,3709,0,0.083808,0.635348
294548,5890,3256,1,2,6,1,2,0,7,0,0,6010,0,0.001091,0.112594


In [55]:
class rec_metrics(object):
    def __init__(self, evaluate_data,k):
        self.users = np.array(evaluate_data['user_id'].to_list())
        self.y_true = np.array(evaluate_data['label'].to_list())
        self.y_pred = np.array(evaluate_data['predict'].to_list())
        self.item_id = np.array(evaluate_data['item_id'].to_list())
        self.item_cate = np.array(evaluate_data['cate'].to_list())
        self.item_pop = np.array(evaluate_data['popularity'].to_list())
        self.k = k
        self.select_cols = [self.y_true,self.y_pred,self.item_id,self.item_cate,self.item_pop]
    def get_user_pred(self):
        """
        divide the result into different group by user id

        Args:
        y_true: array, all true labels of the data
        y_pred: array, the predicted score
        users: array, user id

        Return:
        user_pred: dict, key is user id and value is the labels and scores of each user
        """
        user_pred = {}
        for i, u in enumerate(self.users):
            if u not in user_pred:
                # user_pred[u] = [[self.y_true[i]], [self.y_pred[i]],[self.item_id[i]],[self.item_cate[i]]]
                user_pred[u] = [[feat[i]] for feat in self.select_cols]
            else:
                for index in range(len(self.select_cols)):
                    user_pred[u][index].append(self.select_cols[index][i])
                # user_pred[u][0].append(self.y_true[i])
                # user_pred[u][1].append(self.y_pred[i])
                # user_pred[u][2].append(self.item_id[i])
                # user_pred[u][3].append(self.item_cate[i])
        return user_pred
    def get_user_topk(self):
        
        """
        sort y_pred and find topk results
        this function is used to find topk predicted scores
        and the corresponding index is applied to find the corresponding labels

        """
        user_pred = self.get_user_pred()
        for u in user_pred:
            idx = np.argsort(user_pred[u][1])[::-1][:self.k]
            for i in range(len(self.select_cols)):
                user_pred[u][i] = np.array(user_pred[u][i])[idx]
        return user_pred

    def auc_score(self):
        return roc_auc_score(self.y_true, self.y_pred)
    
    def gauc_score(self,weights=None):
        """
        Args:
        y_true: array, dim(N, ), all true labels of the data
        y_pred: array, dim(N, ), the predicted score
        users: array, dim(N, ), user id
        weight: dict, it contains weights for each group.
            if it is None, the weight is equal to the number
            of times the user is recommended
        Return:
        score: float, GAUC
        """
        assert len(self.y_true) == len(self.y_pred) and len(self.y_true) == len(self.users)

        user_pred = self.get_user_topk()
        score = 0
        num = 0
        for u in user_pred.keys():
            auc = auc_score(user_pred[u][0], user_pred[u][1])
            if weights is None:
                wg = len(user_pred[u][0])
            else:
                wg = weights[u]
            auc *= wg
            num += wg
            score += auc
        return score / num
    
    def log_loss(self):
        score = self.y_true * np.log(self.y_pred) + (1 - self.y_true) * np.log(1 - self.y_pred)
        return -score.sum() / len(self.y_true)
    
    def ndcg_score(self,user_pred):
        """compute NDCG
        Args:
        user_pred: dict, computed by get_user_topk()
        """
        rank = np.arange(1, self.k+1, 1)
        idcgs = 1. / np.log2(rank + 1)
        idcg = sum(idcgs)
        score = 0
        for u in user_pred:
            dcgs = idcgs[np.where(user_pred[u][0] == 1)]
            dcg = sum(dcgs)
            score += dcg / idcg
        return score / len(user_pred.keys())
    
    def cate_diversity_score(self,user_pred):
        score = 0
        for u in user_pred:
            item_list = user_pred[u][3]     #bf : item_id 2. ml-1m: genres 3
            # item_list = sum(item_list,[])
            cate = set(item_list)
            cate_num = len(cate)
            score += cate_num
        return score / len(user_pred.keys())
    
    def popularity_score(self,user_pred):
        score = 0
        for u in user_pred:
            pop_list = user_pred[u][4]
            pop_score = np.mean(pop_list)
            score += pop_score
        return score / len(user_pred.keys())
    
    def hit_score(self,user_pred):
        score = 0
        for u in user_pred:
            if 1 in user_pred[u][0]:
                score += 1.0
        return score / len(user_pred.keys())
    def mrr_score(self,user_pred):
        score = 0
        for u in user_pred:
            if 1 in user_pred[u][0]:
                score += 1.0 / (np.where(user_pred[u][0] == 1)[0][0] + 1)
        return score / len(user_pred.keys())
    def recall_score(self,user_pred):
        score = 0
        for u in user_pred:
            score += sum(user_pred[u][0]) * 1. / len(user_pred[u][0])
        return score / len(user_pred.keys())
    
    def get_all_metrics(self):
        user_pred = self.get_user_topk()
        
        log_loss = self.log_loss()
        auc_score = self.auc_score()
        # gauc_score = self.gauc_score()
        ndcg_score = self.ndcg_score(user_pred)
        hit_score = self.hit_score(user_pred)
        mrr_score = self.mrr_score(user_pred)
        recall_score = self.recall_score(user_pred)
        cate_diversity_score = self.cate_diversity_score(user_pred)
        pop_score = self.popularity_score(user_pred)
        
        result = pd.DataFrame()
        result['log_loss'] = [log_loss]
        result['auc'] = [auc_score]
        result['ndcg'] = [ndcg_score]
        result['hit'] = [hit_score]
        result['mrr'] = [mrr_score]
        result['cate_diversity'] = [cate_diversity_score]
        result['popularity'] = [pop_score]
        return result
    
class rec_fairness_metrics(object):
    def __init__(self, group_target, group_compare):
        self.group_target = group_target
        self.group_compare = group_compare
        self.label_column = 'label'
        self.pred_column = 'predict'
        self.item_id = group_target['item_id'].unique()
        
        self.group_target_metrics = rec_metrics(group_target,5)
        self.group_compare_metrics = rec_metrics(group_compare,5)
    
    def sub_pred_true(self,data):
        y_true = np.array(data[self.label_column].to_list())
        y_pred = np.array(data[self.pred_column].to_list())
        result = np.mean(y_pred - y_true)
        return result
        
    def value_unfairness(self,select_group,compare_group):
        item_id = self.item_id
        val_unfairness_list = []
        for i in item_id:
            user_select_group = select_group.loc[select_group['item_id'] == i]
            if len(user_select_group) == 0:
                val_target = 0
            else:
                val_target = self.sub_pred_true(user_select_group)
            user_compare_group = compare_group.loc[compare_group['item_id'] == i]
            if len(user_compare_group) == 0:
                val_compare = 0
            else:
                val_compare = self.sub_pred_true(user_compare_group)
            val_unfairness_list.append(val_target - val_compare)
        return(np.mean(val_unfairness_list))
    
    def parity_unfairness(self,select_group,compare_group):
        item_id = self.item_id
        for i in item_id:
            user_select_group = select_group.loc[select_group['item_id'] == i]
            if len(user_select_group) == 0:
                par_target = 0
            else:
                par_target = np.mean(user_select_group[self.pred_column].to_list())
            user_compare_group = compare_group.loc[compare_group['item_id'] == i]
            if len(user_compare_group) == 0:
                par_compare = 0
            else:
                par_compare = np.mean(user_compare_group[self.pred_column].to_list())
            parity_unfairness = par_target - par_compare
        return parity_unfairness
    
    def metric_unfairness(self):
        metrics_target = self.group_target_metrics.get_all_metrics()
        metrics_compare = self.group_compare_metrics.get_all_metrics()
        metrics_unfairness = metrics_target.to_numpy() - metrics_compare.to_numpy()
        result_unfairness = pd.DataFrame(metrics_unfairness,columns=metrics_target.columns)
        return result_unfairness
    
    def get_all_unfairness_metrics(self):
        result_unfairness = self.metric_unfairness()
        
        select_group = self.group_target
        compare_group = self.group_compare
        value_unfairness = self.value_unfairness(select_group,compare_group)
        parity_unfairness = self.parity_unfairness(select_group,compare_group)
        
        result_unfairness['value'] = value_unfairness
        result_unfairness['parity'] = parity_unfairness
        return result_unfairness
        

In [32]:
#female : 0     male : 1
female = test_data.loc[test_data['gender'] == 0]
male = test_data.loc[test_data['gender'] == 1]

In [56]:
bf_eva = rec_fairness_metrics(female,male)
bf_eva.get_all_unfairness_metrics()

,log_loss,auc,ndcg,hit,mrr,cate_diversity,popularity,value,parity
0,0.02593,-0.006814,-0.006841,-0.019507,-0.020085,0.084605,-0.002753,0.011397,-0.102536


In [46]:
fe_sample = female.loc[female.user_id.isin([0,5])]
fe_sample

,user_id,item_id,gender,age,occupation,city,stay_years,martial_status,cate,subcate_1,subcate_2,price,label,popularity,predict
0,0,2376,0,0,10,0,2,0,0,5,11,15200,1,0.105623,0.847192
5,5,1009,0,5,9,0,1,0,4,13,0,6918,1,0.064901,0.933879
5891,0,3130,0,0,10,0,2,0,10,0,0,7498,0,0.093261,0.563997
5892,0,411,0,0,10,0,2,0,4,17,0,5439,0,0.039995,0.355139
5893,0,2433,0,0,10,0,2,0,4,12,0,7194,0,0.017634,0.232882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6180,5,3058,0,5,9,0,1,0,7,0,0,7928,0,0.007635,0.143095
6181,5,1631,0,5,9,0,1,0,7,12,0,7961,0,0.012544,0.300282
6182,5,2242,0,5,9,0,1,0,7,0,0,7830,0,0.015634,0.621878
6183,5,139,0,5,9,0,1,0,5,7,13,16146,0,0.004363,0.047314


In [50]:
fe_sample = female.loc[female.user_id.isin([0,5])]
fe_eva = rec_metrics(fe_sample,5)
s = fe_eva.get_user_topk()
ma_eva = rec_metrics(male,5)
ma_eva.get_all_metrics()